### auxfunc.ipynb
- Siep Dokter
- Emil Jousimaa
- Oleksandr Sosovskyy
- Mario Gabriele Carofano

> This file contains all the auxiliary functions used in the other .ipynb files of the repository

In [ ]:
# IMPORTS
import import_ipynb
import constants
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plot

In [ ]:
def get_anchors_name(directory_path):
    '''
    Examine the dataset at 'directory_path' in order to get the anchor names.

    Parameters:
    directory_path (str): The path of the directory which contains the dataset.

    Returns:
    anchors (list): Returns a list which values are the anchors' name.
    '''

    anchors = []
    
    for path in os.listdir(directory_path):
        if path.lower().endswith(".csv"):
            anchors.append(path)

    anchors.sort()

    return anchors

# https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory
# https://stackoverflow.com/questions/5899497/how-can-i-check-the-extension-of-a-file

In [ ]:
def scan_dataset(dirname):
    '''
    Scan the 'dirname' directory in order to get all the subdirectories' path.

    Parameters:
    dirname (str) : The directory's path to scan.

    Returns:
    subfolders (list) : Returns a list containing all the subdirectories of 'dirname'.
    '''

    subfolders = [
        f.path
        for f in os.scandir(dirname)
        if f.is_dir()
        and os.listdir(f.path)
    ]

    for dirname in list(subfolders):
        subfolders.extend(scan_dataset(dirname))

    return subfolders

# https://stackoverflow.com/questions/973473/getting-a-list-of-all-subdirectories-in-the-current-directory
# https://stackoverflow.com/questions/49284015/how-to-check-if-folder-is-empty-with-python

In [ ]:
def define_dataset(scenario_name):
    '''
    Creates the dictionary employed by the estimators to store all the salient information
    about their execution. This dictionary has the following structure:
    -   A key for the name of the scenario.
    -   A key for the type of the anchors.
    -   A key for the transmission's technology.
    -   A key for the actual values, such as "Anchors' Name" and "Dataframes"
    -   A key for the estimated values,
        such as the matrix and the vectors composing the system of equations to be solved.
    
    Parameters:
    scenario_name (str) : The name of the scenario to be studied.

    Returns:
    dataset (dict) : Returns a dictionary whose structure is explained above.
    '''
    
    dataset = {}
    
    directory_list = scan_dataset(constants.DATASET_DIRECTORY)
    for index in range(len(directory_list)):
        directory_list[index] = directory_list[index].replace(constants.DATASET_DIRECTORY, "")
        index = index + 1

    for dirname in directory_list:
        if scenario_name in dirname:

            p = dataset
            for x in dirname.split('/'):
                p = p.setdefault(x, {})

            anchors = get_anchors_name(constants.DATASET_DIRECTORY + dirname)
            n_anchors = len(anchors)

            if n_anchors > 0:
                p.update([("Actual", {}), ("Estimated", {})])
                p["Actual"]["Dataframes"] = list()
                p["Actual"]["Anchors' Name"] = list(anchors)
    
    dataset[scenario_name]["RPI"].update([(constants.HYBRID_TECHNOLOGY, {})])
    dataset[scenario_name]["RPI"][constants.HYBRID_TECHNOLOGY].update([("Actual", {}), ("Estimated", {})])

    hybrid_anchors = []
    for dirname in directory_list:
        if scenario_name in dirname and "RPI" in dirname:
            anchors = get_anchors_name(constants.DATASET_DIRECTORY + dirname)
            hybrid_anchors = hybrid_anchors + anchors

    dataset[scenario_name]["RPI"][constants.HYBRID_TECHNOLOGY]["Actual"]["Dataframes"] = list()
    dataset[scenario_name]["RPI"][constants.HYBRID_TECHNOLOGY]["Actual"]["Anchors' Name"] = hybrid_anchors

    return dataset

# https://stackoverflow.com/questions/9618862/how-to-parse-a-directory-structure-into-dictionary
# https://www.digitalocean.com/community/tutorials/python-add-to-dictionary#add-to-python-dictionary-using-the-update-method


In [ ]:
def calculate_smallest_dataset(dataframes):
    """
    Calculates the minimum over all dataframes' length.

    Parameters:
    dataframes (list): a list which values are the anchors' dataframe.

    Returns:
    minimum_length (int): an integer which value represents the minimum over all dataframes' length.
    """

    sizes = []

    for anchor in range(len(dataframes)):
        df = dataframes[anchor]
        sizes.append(len(df))

    return min(sizes)

In [ ]:
def calculate_burst_quantity(length):
    """
    Calculates the amount of bursts over all dataframes.

    Parameters:
    length (int): An integer which value represents the number of data to read in the dataset.

    Returns:
    bursts (int): An integer which value represents the amount of bursts / configurations.
    """

    bursts = length // constants.BURST_SIZE
    end = constants.BURST_SIZE * bursts

    if (end < length):
        bursts = bursts + 1
    
    return bursts

In [ ]:
def calculate_average_RSS(power_values, configuration, length):
    """
    Calculates the average RSS value over a burst of 'constants.BURST_SIZE' consecutive measurements.
    This is the implementation of (3) on the reference paper.

    Parameters:
    power_values (list) : A list containing RSS values of a specific anchor.
    configuration (int) : An integer representing the selected burst of measurements.
    length (int) : An integer which value represents the number of data to read in the dataset.

    Returns:
    Returns the average of the input RSS values.
    """

    start = configuration * constants.BURST_SIZE

    end = (configuration + 1) * constants.BURST_SIZE
    if length < end:
        end = length

    return np.mean(power_values[start:end])

In [ ]:
def calculate_target_anchor_estimation(average_RSS):
	"""
	Calculates the distance estimates between one anchor and the target, using the average RSS value.
	This is the implementation of (4) on the reference paper.

	Parameters:
	average_RSS (float): The average_RSS over a burst of 'constants.BURST_SIZE' consecutive measurements.

	Returns:
	Returns the distance estimation.
	"""

	num = average_RSS - constants.REFERENCE_POWER
	den = -10 * constants.PATH_LOSS_EXPONENT

	return 10**(num/den)

In [ ]:
def calculate_rmse(data):
	"""
	Calculate the Root Mean Square Error (RMSE) between actual and estimated positions.

	Parameters:
	data (dict): Dictionary containing 'Actual' and 'Estimated' keys, each with a 'Target Coordinates' key.

	Returns:
	rmse (float): Root Mean Square Error.
	"""
	
	# print("data: ", data)
	# Estrai la posizione reale
	actual_position = np.array(data["Actual"]["Target Coordinates"])
	
	# print("actual position: ", actual_position)
	# Estrai le posizioni stimate
	estimated_positions = np.array([estimation["Target Coordinates"] for estimation in data["Estimated"].values()])

	# print("estimated posizions: ", estimated_positions)
	# Calcola gli errori quadrati
	errors_squared = np.sum((actual_position - estimated_positions)**2, axis=1)

	# Calcola la media degli errori quadrati
	mean_squared_error = np.mean(errors_squared)

	# Calcola l'RMSE
	rmse = np.sqrt(mean_squared_error)

	return round(rmse, 3)

In [ ]:
def plot_data(estimator_type, estimator_dict):
	'''
	Plot the actual target and anchors' coordinates, and the estimated ones
	for each communication technology, for each devices' type and for each scenario.

	Parameters:
	estimator_type (str) : The name of the estimator, shown at the top of the plot.
	estimator_dict (dict) : A dictionary containing all the values to plot.

	Returns:
	None
	'''

	for scenario in estimator_dict.keys():
		for type in estimator_dict[scenario].keys():
			xlim = []; ylim = []
			for tech in estimator_dict[scenario][type].keys():
				for anchor_coords in estimator_dict[scenario][type][tech]["Actual"]["Anchor Coordinates"]:
					xlim.append(anchor_coords[0])
					ylim.append(anchor_coords[1])
		
			x_min = min(xlim)
			if x_min < 0:
				x_min = x_min - 2
			else:
				x_min = x_min + 2

			x_max = max(xlim)
			if x_max < 0:
				x_max = x_max - 2
			else:
				x_max = x_max + 2

			y_min = min(ylim)
			if y_min < 0:
				y_min = y_min - 2
			else:
				y_min = y_min + 2

			y_max = max(ylim)
			if y_max < 0:
				y_max = y_max - 2
			else:
				y_max = y_max + 2

			# Calculate the size of the plot
			# print("x: ", x_min, x_max, "; y: ", y_min, y_max)
			plot_height = constants.PLOT_WIDTH * ((y_max - y_min) / (x_max - x_min))

			out_directory = constants.OUTPUT_DIRECTORY + estimator_type + "/"
			out_name = estimator_type + "_" + scenario + "_" + type
			title = out_name + " - " + "Actual vs. Estimated Target Coordinates"

			os.makedirs(out_directory, exist_ok=True)

			plot.figure(num=scenario+type, figsize=[constants.PLOT_WIDTH, plot_height])
			plot.title(title)
			plot.xlim(x_min, x_max)
			plot.ylim(y_min, y_max)

			plot_colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
			tech_index = 1
			for tech in estimator_dict[scenario][type].keys():
				legend_index = 1

				for configuration in estimator_dict[scenario][type][tech]["Estimated"].values():
					target_estimation = configuration["Target Coordinates"]
					if legend_index == 1:
						label = type + "-" + tech
						color_index = tech_index % len(plot_colors)
						format = plot_colors[color_index] + '.'
						# print(label, legend_index, color_index, format)
						plot.plot(target_estimation[0], target_estimation[1], format, label=label)
					else:
						pass
						plot.plot(target_estimation[0], target_estimation[1], format)
					legend_index = legend_index + 1
				tech_index = tech_index + 1

			target_actual = estimator_dict[scenario]["RPI"][constants.BLT_TECHNOLOGY]["Actual"]["Target Coordinates"]
			plot.plot(target_actual[0], target_actual[1], 'k*', label="Target")
			# plot.text(target_actual[0], target_actual[1], key, fontsize='xx-small', horizontalalignment='center')

			legend_index = 1
			for tech in estimator_dict[scenario][type].keys():
				for anchor_coords in estimator_dict[scenario][type][tech]["Actual"]["Anchor Coordinates"]:
					if legend_index == 1:
						plot.plot(anchor_coords[0], anchor_coords[1], 'ks', label="Anchors")
					else:
						plot.plot(anchor_coords[0], anchor_coords[1], 'ks')
					legend_index = legend_index + 1

			
			plot.legend()
			plot.savefig(out_directory + out_name + ".pdf", bbox_inches='tight')
			plot.show()
	
	return

# https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html
# https://stackoverflow.com/questions/28504737/how-to-plot-a-single-point-in-matplotlib
# https://www.scaler.com/topics/matplotlib/matplotlib-set-axis-range/
# https://stackoverflow.com/questions/9622163/save-plot-to-image-file-instead-of-displaying-it
# https://note.nkmk.me/en/python-os-mkdir-makedirs/